In [1]:
import tensorflow as tf
import model

ImportError: Traceback (most recent call last):
  File "/home/aagusev2/anaconda3/envs/tf_env/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/aagusev2/anaconda3/envs/tf_env/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/aagusev2/anaconda3/envs/tf_env/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/aagusev2/anaconda3/envs/tf_env/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/home/aagusev2/anaconda3/envs/tf_env/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: libcuda.so.1: cannot open shared object file: N

In [ ]:
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf

from config import Config
from interactive_predict import InteractivePredictor
from model import Model

    
parser = ArgumentParser()
parser.add_argument('--release', action='store_true',
                    help='if specified and loading a trained model, release the loaded model for a smaller model '
                         'size.')

args = parser.parse_args()

if args.debug:
    config = Config.get_debug_config(args)
else:
    config = Config.get_default_config(args)

model = Model(config)
print('Created model')
if config.TRAIN_PATH:
    model.train()
if config.TEST_PATH and not args.data_path:
    results, precision, recall, f1, rouge = model.evaluate()
    print('Accuracy: ' + str(results))
    print('Precision: ' + str(precision) + ', recall: ' + str(recall) + ', F1: ' + str(f1))
    print('Rouge: ', rouge)
if args.predict:
    predictor = InteractivePredictor(config, model)
    predictor.predict()
if args.release and args.load_path:
    model.evaluate(release=True)
    print(model.index_to_node)
model.close_session()



In [4]:
! echo $PATH

/home/aagusev2/anaconda3/envs/tf_env/bin:/home/aagusev2/anaconda3/condabin:/home/aagusev2/anaconda3/bin:/opt/ohpc/pub/mpi/openmpi3-gnu7/3.1.3/bin:/opt/ohpc/pub/compiler/gcc/7.3.0/bin:/opt/ohpc/pub/utils/prun/1.3:/opt/ohpc/pub/utils/autotools/bin:/opt/ohpc/pub/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/ibutils/bin:/opt/pbs/bin:/home/aagusev2/.local/bin:/home/aagusev2/bin


In [ ]:
python extract_input_words.py --load models/java-small-model/2020_04_21__1e-05_0.0001_0.00316__eda7e095/model_iter41 --release